In [1]:
import pandas as pd


DAQUAR_PATH = "../data/raw/daquar/data_train.csv"

daquar_train = pd.read_csv(DAQUAR_PATH)
daquar_train.head()

,question,answer,image_id
0,what is on the right side of the black telepho...,desk,image3
1,what is in front of the white door on the left...,telephone,image3
2,what is on the desk,"book, scissor, papers, tape_dispenser",image3
3,what is the largest brown objects,carton,image3
4,what color is the chair in front of the white ...,red,image3


In [2]:
daquar_train.count()

question    6795
answer      6795
image_id    6795
dtype: int64

In [3]:
multiple_answers = daquar_train[daquar_train["answer"].str.contains(",")]
multiple_answers.head()

,question,answer,image_id
2,what is on the desk,"book, scissor, papers, tape_dispenser",image3
14,what is in front of the white board or in fron...,"table, chair",image5
26,what are on the wall on the left side of the g...,"clock, telephone",image8
72,what are the white objects on the mattress,"blanket, pillow",image73
74,what color is the hat on the top right of the ...,"black, white",image80


In [4]:
multiple_answers.count()

question    646
answer      646
image_id    646
dtype: int64

In [5]:
new_df = daquar_train.iloc[daquar_train.index.repeat(daquar_train["answer"].str.count(",") + 1)]
new_df.head()

,question,answer,image_id
0,what is on the right side of the black telepho...,desk,image3
1,what is in front of the white door on the left...,telephone,image3
2,what is on the desk,"book, scissor, papers, tape_dispenser",image3
2,what is on the desk,"book, scissor, papers, tape_dispenser",image3
2,what is on the desk,"book, scissor, papers, tape_dispenser",image3


In [6]:
split_answer = daquar_train["answer"].str.split(",", expand=True).stack().reset_index(level=1, drop=True).rename("answer")
split_answer

0             desk
1        telephone
2             book
2          scissor
2           papers
           ...    
6791        window
6792       cabinet
6793    candelabra
6793          book
6794         chair
Name: answer, Length: 7768, dtype: object

In [7]:
new_df.loc[:, "answer"] = split_answer.str.strip()
new_df.reset_index(drop=True, inplace=True)
new_df.head()

/tmp/ipykernel_5590/1585816665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[:, "answer"] = split_answer.str.strip()


,question,answer,image_id
0,what is on the right side of the black telepho...,desk,image3
1,what is in front of the white door on the left...,telephone,image3
2,what is on the desk,book,image3
3,what is on the desk,scissor,image3
4,what is on the desk,papers,image3


In [8]:
from pathlib import Path
from datasets import load_dataset

PROCESSED_DAQUAR_PATH = Path("../data/processed/daquar")

daquar_dataset = load_dataset(
        "csv",
        data_files={
            "train": str(PROCESSED_DAQUAR_PATH / "daquar_train_flattened.csv"),
            "validation": str(PROCESSED_DAQUAR_PATH / "daquar_eval_flattened.csv"),
        },
    )
daquar_dataset

Found cached dataset csv (/home/vlad/.cache/huggingface/datasets/csv/default-955a7bafb90eef83/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'image_id'],
        num_rows: 7768
    })
    validation: Dataset({
        features: ['question', 'answer', 'image_id'],
        num_rows: 6562
    })
})

In [9]:
from utils.datasets.daquar import load_daquar

daquar = load_daquar()
daquar["train"][:10]

Found cached dataset csv (/home/vlad/.cache/huggingface/datasets/csv/default-955a7bafb90eef83/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/7768 [00:00<?, ? examples/s]

Map:   0%|          | 0/6562 [00:00<?, ? examples/s]

{'question': ['what is on the right side of the black telephone and on the left side of the red chair',
  'what is in front of the white door on the left side of the desk',
  'what is on the desk',
  'what is on the desk',
  'what is on the desk',
  'what is on the desk',
  'what is the largest brown objects',
  'what color is the chair in front of the white wall',
  'what is on the right side of the notebook on the desk',
  'what is on the right and left and in front of the papers on the desk'],
 'answer': ['desk',
  'telephone',
  'book, scissor, papers, tape_dispenser',
  'book, scissor, papers, tape_dispenser',
  'book, scissor, papers, tape_dispenser',
  'book, scissor, papers, tape_dispenser',
  'carton',
  'red',
  'plastic_cup_of_coffee',
  'notebook'],
 'image_id': ['image3',
  'image3',
  'image3',
  'image3',
  'image3',
  'image3',
  'image3',
  'image3',
  'image4',
  'image4'],
 'image': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=560x425>,
  <PIL.PngImagePlugin